# other aprouch for webscarping 


### Laberies 

In [13]:
pip install selenium


^C
Note: you may need to restart the kernel to use updated packages.


  Using cached websocket_client-1.9.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.6 MB 8.1 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.6 MB 8.1 MB/s eta 0:00:01
   ------------------- -------------------- 4.7/9.6 MB 8.5 MB/s eta 0:00:01
   -------------------------- ------------- 6.3/9.6 MB 8.2 MB/s eta 0:00:01
   ------------------------------ --------- 7.3/9.6 MB 7.9 MB/s eta 0:00:01
   -------------------------------------- - 9.2/9.6 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 7.6 MB/s  0:00:01
Using cached websocket_client-1.9.0-py3-none-any.whl (82 kB)
Using cac


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install python-dotenv


  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import time
import csv
from datetime import datetime
from dotenv import load_dotenv
import os

### agrements / vairbales 

In [2]:
# --- CONFIG ---
JOB_KEYWORD = "Data scientist"
COUNTRIES = ["spain"]
DATE_POSTED = "24h"
EXPERIENCE_LEVELS = ["2"]
WORKPLACE_TYPES = ["2"]

# DATE_POSTED codes:            # EXPERIENCE_LEVELS codes:          # WORKPLACE_TYPES codes:    
# "any" = Any time              # [] = All levels                   # [] = All types       
# "24h" = Past 24 hours         # ["1"] = Internship                # ["1"] = On-site  
# "week" = Past week            # ["2"] = Entry level               # ["2"] = Remote   
# "month" = Past month          # ["3"] = Associate                 # ["3"] = Hybrid   
                                # ["4"] = Mid-Senior level          
                                # ["5"] = Director                  
                                # ["6"] = Executive                 

In [3]:
# --- SCROLL ---
MAX_SCROLL_ATTEMPTS = 200
SCROLL_PAUSE = 5
DETAIL_PAUSE = 2


In [4]:
# --- SETUP CHROME ---
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options)

### Functions 

In [5]:
# --- HELPER FUNCTIONS ---
def build_linkedin_url(keyword, location, exp_levels, workplace_types, date_posted):
    exp_param = ",".join(exp_levels) if exp_levels else ""
    workplace_param = ",".join(workplace_types) if workplace_types else ""
    date_param = ""
    if date_posted == "24h": date_param = "r86400"
    elif date_posted == "week": date_param = "r604800"
    elif date_posted == "month": date_param = "r2592000"

    url = f"https://www.linkedin.com/jobs/search/?keywords={quote_plus(keyword)}&location={quote_plus(location)}"
    if exp_param: url += f"&f_E={exp_param}"
    if workplace_param: url += f"&f_WT={workplace_param}"
    if date_param: url += f"&f_TPR={date_param}"
    url += "&position=1&pageNum=0"
    return url

In [6]:
def scroll_page(driver):
    attempt = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    while attempt < MAX_SCROLL_ATTEMPTS:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE)
        try:
            show_more_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "infinite-scroller__show-more-button"))
            )
            show_more_btn.click()
            time.sleep(SCROLL_PAUSE)
        except:
            pass
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        attempt += 1

In [7]:
def fetch_job_details(job_url):
    job_desc = ""
    company_desc = ""
    if not job_url: 
        return job_desc, company_desc
    try:
        driver.get(job_url)
        time.sleep(DETAIL_PAUSE)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "description__text"))
        )
        job_soup = BeautifulSoup(driver.page_source, "html.parser")
        job_div = job_soup.find("div", class_="description__text")
        job_desc = job_div.get_text(separator="\n", strip=True) if job_div else ""
        company_div = job_soup.find("div", class_="show-more-less-html__markup")
        company_desc = company_div.get_text(separator="\n", strip=True) if company_div else ""
    except Exception as e:
        print(f"⚠️ Failed to fetch job detail: {e}")
    return job_desc, company_desc

In [8]:
# --- MAIN SCRAPING LOOP ---
all_jobs = []

for country in COUNTRIES:
    print(f"=== Scraping LinkedIn Jobs for {country} ===")
    url = build_linkedin_url(JOB_KEYWORD, country, EXPERIENCE_LEVELS, WORKPLACE_TYPES, DATE_POSTED)
    print(f"🔗 URL: {url}")
    driver.get(url)
    scroll_page(driver)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    job_cards = soup.find_all("div", class_="base-card") 
    
    for idx, card in enumerate(job_cards):
        a_tag = card.find("a", class_="base-card__full-link")
        job_url = a_tag["href"].strip() if a_tag else ""
        job_title = a_tag.find("span", class_="sr-only").text.strip() if a_tag and a_tag.find("span", class_="sr-only") else ""
        company_tag = card.find("h4", class_="base-search-card__subtitle")
        company_a = company_tag.find("a") if company_tag else None
        company_name = company_a.text.strip() if company_a else ""
        company_url = company_a["href"].strip() if company_a else ""
        location = card.find("span", class_="job-search-card__location")
        location = location.text.strip() if location else ""
        benefit = card.find("span", class_="job-posting-benefits__text")
        benefit = benefit.text.strip() if benefit else ""
        posted = card.find("time", class_="job-search-card__listdate")
        posted = posted.text.strip() if posted else ""

        print(f"🔍 ({idx + 1}/{len(job_cards)}) Fetching job: {job_title}")
        job_description, company_description = fetch_job_details(job_url)

        all_jobs.append({
            "country": country,
            "job_title": job_title,
            "company_name": company_name,
            "company_url": company_url,
            "location": location,
            "benefit": benefit,
            "posted": posted,
            "company_description": company_description,
            "job_url": job_url,
            "job_description": job_description
        })


driver.quit()

=== Scraping LinkedIn Jobs for spain ===
🔗 URL: https://www.linkedin.com/jobs/search/?keywords=Data+scientist&location=spain&f_E=2&f_WT=2&f_TPR=r86400&position=1&pageNum=0
🔍 (1/17) Fetching job: Junior Data Scientist
🔍 (2/17) Fetching job: Programa Talento Joven - Data Science
🔍 (3/17) Fetching job: Data Scientist (NLP Deep Learning Engineer)
🔍 (4/17) Fetching job: Data Scientist Observability - PhD
🔍 (5/17) Fetching job: Junior Developer - AI, App, Web, Cloud, Data, Machine Learning
🔍 (6/17) Fetching job: Data Analyst/Engineer/Scientist
🔍 (7/17) Fetching job: Puestos vacantes: Consultoría Junior (9451)
🔍 (8/17) Fetching job: Investigador de Aprendizaje Automático
🔍 (9/17) Fetching job: Tècnic Analista Junior
🔍 (10/17) Fetching job: QA Engineer
🔍 (11/17) Fetching job: Junior QA Engineer - (Remote from Spain)
🔍 (12/17) Fetching job: Data Analyst Junior
🔍 (13/17) Fetching job: Data Analyst – Crecimiento dentro del sector tech
🔍 (14/17) Fetching job: Junior Data Analyst
🔍 (15/17) Fetching

In [9]:
all_jobs


[{'country': 'spain',
  'job_title': 'Junior Data Scientist',
  'company_name': 'Joinrs',
  'company_url': 'https://it.linkedin.com/company/joinrs-international?trk=public_jobs_jserp-result_job-search-card-subtitle',
  'location': 'Spain',
  'benefit': 'Actively Hiring',
  'posted': '',
  'company_description': 'Nuestro objetivo es ayudar a todos los recién graduados del mundo a encontrar el trabajo que más se ajuste a sus ambiciones: ¡únete a nosotros para hacer realidad las aspiraciones de una comunidad de más de 1,500,000 usuarios!\nObservamos constantemente los recursos de los estudiantes y egresados que aspiran a incorporarse a nuestro equipo de\nTecnología\n: los candidatos para este puesto y nuestro equipo de RRHH se pondrán en contacto contigo sin comunicar puesto y tu CV será el adecuado a tu perfil.\nPara estas posiciones junior en el sector\nTech\n, ofrecemos un contrato con una\nremuneración anual de 24.000€ a 28.000 €\n, dependiendo de la experiencia. Podrás trabajar en mo

In [13]:
all_jobs[0]["job_url"]




'https://es.linkedin.com/jobs/view/junior-data-scientist-at-joinrs-4373566199?position=1&pageNum=0&refId=%2FKjwEcZtc02n2BegeUOtdg%3D%3D&trackingId=FUxAH%2B6v8LCz3%2FD7CeRO2w%3D%3D'

In [10]:
import pandas as pd


In [12]:
df = pd.DataFrame(all_jobs)
df.head()


,country,job_title,company_name,company_url,location,benefit,posted,company_description,job_url,job_description
0,spain,Data Scientist Intern - Data Platform & AI,Zurich Insurance,https://ch.linkedin.com/company/zurich-insuran...,"Barcelona, Catalonia, Spain",,,We Are Waiting for You\nHi there future intern...,https://es.linkedin.com/jobs/view/data-scienti...,We Are Waiting for You\nHi there future intern...
1,spain,Programa Talento Joven - Data Science,Evolve,https://es.linkedin.com/school/evolve-es/?trk=...,Spain,Be an early applicant,,Descripción de la oferta\nEn nuestro Programa ...,https://es.linkedin.com/jobs/view/programa-tal...,Descripción de la oferta\nEn nuestro Programa ...
2,spain,Data Engineer / Data Scientist,Inetum,https://fr.linkedin.com/company/inetum?trk=pub...,"Madrid, Community of Madrid, Spain",,,Descripción de la empresa\nInetum es líder eur...,https://es.linkedin.com/jobs/view/data-enginee...,Descripción de la empresa\nInetum es líder eur...
3,spain,Machine Learning Engineer - AI Data Trainer,Alignerr,https://www.linkedin.com/company/alignerr?trk=...,Spain,,,"Location: Remote\nAbout The Job\nAt Alignerr, ...",https://es.linkedin.com/jobs/view/machine-lear...,"Location: Remote\nAbout The Job\nAt Alignerr, ..."
4,spain,Junior Data Scientist – Proyectos reales y evo...,Evolve,https://es.linkedin.com/school/evolve-es/?trk=...,Spain,Be an early applicant,,Descripción de la oferta\nEn nuestro Programa ...,https://es.linkedin.com/jobs/view/junior-data-...,Descripción de la oferta\nEn nuestro Programa ...


In [18]:
df.shape

(69, 10)

In [15]:
def filter_data_ai_jobs(df):
    return df[df["job_title"].str.contains("data|ai", case=False, na=False)]


In [16]:
df_filtered = filter_data_ai_jobs(df)


In [17]:
df_filtered.shape


(23, 10)

In [24]:
df_filtered.head()

,country,job_title,company_name,company_url,location,benefit,posted,company_description,job_url,job_description
0,spain,Data Scientist Intern - Data Platform & AI,Zurich Insurance,https://ch.linkedin.com/company/zurich-insuran...,"Barcelona, Catalonia, Spain",,,We Are Waiting for You\nHi there future intern...,https://es.linkedin.com/jobs/view/data-scienti...,We Are Waiting for You\nHi there future intern...
1,spain,Programa Talento Joven - Data Science,Evolve,https://es.linkedin.com/school/evolve-es/?trk=...,Spain,Be an early applicant,,Descripción de la oferta\nEn nuestro Programa ...,https://es.linkedin.com/jobs/view/programa-tal...,Descripción de la oferta\nEn nuestro Programa ...
2,spain,Data Engineer / Data Scientist,Inetum,https://fr.linkedin.com/company/inetum?trk=pub...,"Madrid, Community of Madrid, Spain",,,Descripción de la empresa\nInetum es líder eur...,https://es.linkedin.com/jobs/view/data-enginee...,Descripción de la empresa\nInetum es líder eur...
3,spain,Machine Learning Engineer - AI Data Trainer,Alignerr,https://www.linkedin.com/company/alignerr?trk=...,Spain,,,"Location: Remote\nAbout The Job\nAt Alignerr, ...",https://es.linkedin.com/jobs/view/machine-lear...,"Location: Remote\nAbout The Job\nAt Alignerr, ..."
4,spain,Junior Data Scientist – Proyectos reales y evo...,Evolve,https://es.linkedin.com/school/evolve-es/?trk=...,Spain,Be an early applicant,,Descripción de la oferta\nEn nuestro Programa ...,https://es.linkedin.com/jobs/view/junior-data-...,Descripción de la oferta\nEn nuestro Programa ...


In [21]:
df_filtered_junior = df_filtered[df_filtered["job_title"].str.contains("junior", case=False, na=False)]

In [22]:
df_filtered_junior.shape

(4, 10)